In [17]:
def get_service_code(df):
    service_code_lst = []
    for i in df['campaign_name']:
        if i.find('|') == -1:
            try:
                extract = i.split(' - ')

                service_code_lst.append(extract[1])
            except:
                service_code_lst.append('')
        else:
            extract = i.split(' | ')
            service_code_lst.append(extract[1])
    return service_code_lst

In [18]:
def get_month(df):
    month_lst = []
    for i in df['date']:
        month_lst.append(str(i).split('-')[1])
    return month_lst

In [19]:
def get_from_settings(df):
    client = pygsheets.authorize(service_file = r'C:\Users\Tường\Documents\FACEBOOK-API-BOT--main\maido-agency-e7cc6f08cb95.json')

    sh = client.open('MAIDO - DB MAGIC DATA')
    wks = sh.worksheet_by_title('3. Ads Settings')

    setting_df = wks.get_as_df()
    
    join_df = df.merge(setting_df, how = 'left', left_on = ['service_code'], right_on = ['service_code'])
    cost_ex_lst = []
    for cost,percentage in zip(join_df['cost'], join_df['internal_budget_percent']):
        if percentage == '' and cost =='':
            cost_ex_lst.append('')
        else:
            try:
                cost_ex_lst.append(float(cost)/float(percentage))
            except ZeroDivisionError:
                cost_ex_lst.append(float(cost))
    join_df['cost_external'] = cost_ex_lst

    join_df.drop_duplicates(inplace = True)
    
    return join_df

In [20]:
def get_result(df):
    result_lst = []
    for i in df['campaign_name']:
        if i.find('|') != -1:
            extract = i.split(' | ')[3]
            if 'Mess' in extract:
                result_lst.append('Mess')
            elif 'Engage' in extract:
                result_lst.append('Engage')
            elif 'Page Like' in extract:
                result_lst.append('Page Like')
            else:
                result_lst.append(extract)
        
        else:
            extract = i.split(' - ')
            if len(extract) == 4:
                sub_result = extract[3]
            elif len(extract) == 5:
                if extract[-1] not in ['P1','P2','P3','FOW', 'FOW 1', 'Heart','daily','2tr5','3tr5','3tr750']:
                    sub_result = extract[4]
                else:
                    sub_result = extract[3]
            
            if 'Mess' in sub_result:
                result_lst.append('Mess')
            elif 'Engage' in sub_result:
                result_lst.append('Engage')
            elif 'Page Like' in sub_result:
                result_lst.append('Page Like')
            else:
                result_lst.append(sub_result)
    
    return result_lst


In [21]:
def get_page_data(df):
    client = pygsheets.authorize(service_file = r'C:\Users\Tường\Documents\FACEBOOK-API-BOT--main\maido-agency-e7cc6f08cb95.json')

    sh = client.open('MAIDO - DB MAGIC DATA')
    wks = sh.worksheet_by_title('2. Data Raw - FB Page')

    page_df = wks.get_as_df()

    df['date'] = [str(i).split(' ')[0] for i in df['date']]
    df['join_date'] = [date_+' '+page_name for date_, page_name in zip(df['date'],df['page_name'])]
    page_df['index'] = [1]*len(page_df)

    index = []
    counts = {}
    for i in df['join_date']:
        try:
            counts[i] = counts[i] + 1
            index.append(counts[i])
        except KeyError:
            counts[i] = 1
            index.append(1)

    df.drop('join_date', inplace = True,axis =1)
    df['index'] = index


    join_df = df.merge(page_df, how = 'left', left_on = ['date','page_name','index'], right_on = ['date','page_name','index'])

    join_df.drop_duplicates(inplace = True)


    return join_df